In [1]:
import pandas as pd
import numpy as np
import matplotlib.patches as mpatches
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy import stats
from scipy.interpolate import make_interp_spline
import seaborn as sns; sns.set_theme(color_codes=True)
import os
import matplotlib.pyplot as plt
import sklearn.cluster
import sklearn.metrics
import sklearn.datasets
import random
import warnings
warnings.filterwarnings("ignore")

In [2]:
import matplotlib
matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42
sns.set_style(style='white')

### Load in TAD Data

In [3]:
common_TADs_meQTLs = pd.read_csv("../common_TADs_proper_annot.csv")
common_TADs_meQTLs

,Unnamed: 0,chr1,x1,x2,Label
0,0,1,7295000,7710000,Inactive
1,1,1,7710000,7712500,Boundary-Inactive
2,2,1,7712500,7715000,Boundary-Inactive
3,3,1,7715000,8010000,Inactive
4,4,1,8010000,8020000,Boundary-Inactive
...,...,...,...,...,...
3229,3229,X,133535000,133635000,Boundary-Inactive
3230,3230,X,133635000,133905000,Inactive
3231,3231,X,133905000,134005000,Boundary-Inactive
3232,3232,X,149010000,149110000,Boundary-Inactive


In [4]:
common_TADs_meQTLs = common_TADs_meQTLs.sort_values(["chr1", "x1"])
common_TADs_meQTLs

,Unnamed: 0,chr1,x1,x2,Label
0,0,1,7295000,7710000,Inactive
1,1,1,7710000,7712500,Boundary-Inactive
2,2,1,7712500,7715000,Boundary-Inactive
3,3,1,7715000,8010000,Inactive
4,4,1,8010000,8020000,Boundary-Inactive
...,...,...,...,...,...
3229,3229,X,133535000,133635000,Boundary-Inactive
3230,3230,X,133635000,133905000,Inactive
3231,3231,X,133905000,134005000,Boundary-Inactive
3232,3232,X,149010000,149110000,Boundary-Inactive


In [5]:
common_TADs_meQTLs_noboundary = common_TADs_meQTLs[~common_TADs_meQTLs["Label"].str.contains("Boundary")]
common_TADs_meQTLs_noboundary = common_TADs_meQTLs_noboundary.reset_index(drop=True)
common_TADs_meQTLs_noboundary

,Unnamed: 0,chr1,x1,x2,Label
0,0,1,7295000,7710000,Inactive
1,3,1,7715000,8010000,Inactive
2,6,1,8030000,8410000,Inactive
3,9,1,9170000,9295000,Mixed
4,12,1,9330000,9550000,Inactive
...,...,...,...,...,...
1095,3221,X,108950000,109125000,Inactive
1096,3224,X,109150000,109385000,Inactive
1097,3227,X,117750000,117955000,Mixed
1098,3230,X,133635000,133905000,Inactive


In [6]:
all_labels = dict()
for ch in list(pd.unique(common_TADs_meQTLs_noboundary["chr1"])):
        all_labels[ch] = []
for i, row in common_TADs_meQTLs_noboundary.iterrows():
    if i<len(common_TADs_meQTLs_noboundary)-1:
        if row["chr1"] == common_TADs_meQTLs_noboundary.iloc[i+1]["chr1"]:
            all_labels[row["chr1"]].append(row["Label"]+"-"+common_TADs_meQTLs_noboundary.iloc[i+1]["Label"])
print(all_labels)

{'1': ['Inactive-Inactive', 'Inactive-Inactive', 'Inactive-Mixed', 'Mixed-Inactive', 'Inactive-Active', 'Active-Inactive', 'Inactive-Mixed', 'Mixed-Active', 'Active-Mixed', 'Mixed-Mixed', 'Mixed-Inactive', 'Inactive-Inactive', 'Inactive-Active', 'Active-Inactive', 'Inactive-Inactive', 'Inactive-Inactive', 'Inactive-Inactive', 'Inactive-Inactive', 'Inactive-Inactive', 'Inactive-Inactive', 'Inactive-Active', 'Active-Inactive', 'Inactive-Mixed', 'Mixed-Inactive', 'Inactive-Active', 'Active-Mixed', 'Mixed-Mixed', 'Mixed-Inactive', 'Inactive-Active', 'Active-Mixed', 'Mixed-Inactive', 'Inactive-Inactive', 'Inactive-Inactive', 'Inactive-Active', 'Active-Mixed', 'Mixed-Active', 'Active-Active', 'Active-Inactive', 'Inactive-Mixed', 'Mixed-Active', 'Active-Mixed', 'Mixed-Inactive', 'Inactive-Inactive', 'Inactive-Inactive', 'Inactive-Inactive', 'Inactive-Inactive', 'Inactive-Active', 'Active-Inactive', 'Inactive-Active', 'Active-Mixed', 'Mixed-Active', 'Active-Inactive', 'Inactive-Inactive', 'Ina

In [19]:
print(all_labels.keys())

dict_keys(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '3', '4', '5', '6', '7', '8', '9', 'X'])


In [8]:
active_tads = common_TADs_meQTLs[common_TADs_meQTLs["Label"] == "Active"]
inactive_tads = common_TADs_meQTLs[common_TADs_meQTLs["Label"] == "Inactive"]
mean_bp_active = np.mean(active_tads["x2"]-active_tads["x1"])
mean_bp_inactive = np.mean(inactive_tads["x2"]-inactive_tads["x1"])
print(np.median(active_tads["x2"]-active_tads["x1"]), np.min(active_tads["x2"]-active_tads["x1"]), np.max(active_tads["x2"]-active_tads["x1"]))
print(np.median(inactive_tads["x2"]-inactive_tads["x1"]), np.min(inactive_tads["x2"]-inactive_tads["x1"]), np.max(inactive_tads["x2"]-inactive_tads["x1"]))

print(np.mean(active_tads["x2"]-active_tads["x1"]), np.std(active_tads["x2"]-active_tads["x1"]))
print(np.mean(inactive_tads["x2"]-inactive_tads["x1"]), np.std(inactive_tads["x2"]-inactive_tads["x1"]))

210000.0 130000 620000
235000.0 115000 1050000
223761.26126126127 64911.03915217059
261405.75079872203 118024.96716490356


### Load in meQTL Data

In [10]:
meqtls = pd.read_csv("../unique_meqtls.csv")
meqtls

,Unnamed: 0,cancer_type,rsid,snp_position,alleles,probes,probe_position,probe_gene,beta,status,r,p-value,cancer,chr,bp,SNP
0,0,BLCA,rs11684598,chr2:33952621,G/A,cg04131969,chr2:33951647,MYADML,-1.24,-40.26,-0.90,3.940000e-139,BLCA,chr2,33952621,2:33952621:G:A
1,1,BLCA,rs12232965,chr2:33954560,C/T,cg04131969,chr2:33951647,MYADML,-1.24,-39.59,-0.90,6.710000e-137,BLCA,chr2,33954560,2:33954560:C:T
2,2,BLCA,rs7574695,chr2:33953186,C/T,cg04131969,chr2:33951647,MYADML,-1.24,-39.35,-0.90,4.230000e-136,BLCA,chr2,33953186,2:33953186:C:T
3,3,BLCA,rs11777332,chr8:91676709,G/A,cg16814680,chr8:91681699,NaN,-1.20,-36.65,-0.88,8.170000e-127,BLCA,chr8,91676709,8:91676709:G:A
4,4,BLCA,rs4332092,chr8:91677926,G/A,cg16814680,chr8:91681699,NaN,-1.20,-36.65,-0.88,8.170000e-127,BLCA,chr8,91677926,8:91677926:G:A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1236137,1236137,UCEC,rs60260871,chr3:124711371,C/T,cg11029367,chr3:124705145,HEG1,0.43,4.58,0.35,9.990000e-06,UCEC,chr3,124711371,3:124711371:C:T
1236138,1236138,UCEC,rs57574713,chr3:124711386,G/A,cg11029367,chr3:124705145,HEG1,0.43,4.58,0.35,9.990000e-06,UCEC,chr3,124711386,3:124711386:G:A
1236139,1236139,UCEC,rs10903612,chr10:2018353,C/T,cg16296438,chr10:1416569,ADARB2,0.36,4.58,0.35,9.990000e-06,UCEC,chr10,2018353,10:2018353:C:T
1236140,1236140,UCEC,rs4880953,chr10:2019373,C/T,cg16296438,chr10:1416569,ADARB2,0.36,4.58,0.35,9.990000e-06,UCEC,chr10,2019373,10:2019373:C:T


In [11]:
clumped_meqtls = pd.read_csv("../all_meqtls.clumped", delim_whitespace=True)
clumped_meqtls["CHR"] = clumped_meqtls["CHR"].astype(str)
clumped_meqtls

,CHR,F,SNP,BP,P,TOTAL,NSIG,S05,S01,S001,S0001,SP2
0,9,1,9:37136359:C:G,37136359,2.610000e-186,84,0,0,0,0,84,"9:37070209:C:T(1),9:37070374:G:A(1),9:37072434..."
1,4,1,4:60004266:T:C,60004266,4.220000e-159,262,0,0,0,0,262,"4:59978132:A:G(1),4:59978891:A:T(1),4:59983686..."
2,2,1,2:239335473:T:C,239335473,7.620000e-151,33,0,0,0,0,33,"2:239318389:T:C(1),2:239320505:A:G(1),2:239323..."
3,1,1,1:156165290:T:G,156165290,1.920000e-147,63,0,0,0,0,63,"1:156156094:G:A(1),1:156156789:A:G(1),1:156156..."
4,2,1,2:33952621:G:A,33952621,3.940000e-139,77,0,0,0,0,77,"2:33940162:C:G(1),2:33940918:G:A(1),2:33941247..."
...,...,...,...,...,...,...,...,...,...,...,...,...
60599,4,1,4:4795176:T:C,4795176,9.990000e-06,0,0,0,0,0,0,NONE
60600,6,1,6:23487567:T:C,23487567,9.990000e-06,0,0,0,0,0,0,NONE
60601,7,1,7:154416644:A:G,154416644,9.990000e-06,0,0,0,0,0,0,NONE
60602,10,1,10:31390127:A:G,31390127,9.990000e-06,0,0,0,0,0,0,NONE


In [12]:
commonTADs_short = pd.DataFrame()
commonTADs_short["chr"] = common_TADs_meQTLs["chr1"]
commonTADs_short["x1"] = common_TADs_meQTLs["x1"]
commonTADs_short["x2"] = common_TADs_meQTLs["x2"]
commonTADs_short["label_name"] = common_TADs_meQTLs["Label"]
commonTADs_short

,chr,x1,x2,label_name
0,1,7295000,7710000,Inactive
1,1,7710000,7712500,Boundary-Inactive
2,1,7712500,7715000,Boundary-Inactive
3,1,7715000,8010000,Inactive
4,1,8010000,8020000,Boundary-Inactive
...,...,...,...,...
3229,X,133535000,133635000,Boundary-Inactive
3230,X,133635000,133905000,Inactive
3231,X,133905000,134005000,Boundary-Inactive
3232,X,149010000,149110000,Boundary-Inactive


### Random Shuffle of TAD Labels to Test Uniqueness of meQTL Distributions

In [ ]:
def shuffle_tad_labels(tads):
    rand_labels = tads.copy()
    labels = []
    for ch in all_labels:
        tad_ch = rand_labels[rand_labels["chr1"] == ch]
        rand_categories = all_labels[ch].copy()
        random.shuffle(rand_categories)
        random_labels = []
        for j in range(len(rand_categories)):
            cat = rand_categories[j]
            lst = cat.split("-")
            if j == len(rand_categories)-1:
                random_labels.append(lst[0])
                random_labels.append(lst[1])
            else:
                random_labels.append(lst[0])
        ind = 0
        for i, row in tad_ch.iterrows():
            if "Boundary" in row["Label"]:
                labels.append("Boundary")
            else:
                labels.append(random_labels[ind])
                ind += 1
    rand_labels["Label"] = labels
    return rand_labels

In [13]:
def count_meqtls(chrom, x1, x2):
    common_chr = meqtls[meqtls["chr"] == "chr"+chrom]
    snps = common_chr[(common_chr["bp"] >= x1) & (common_chr["bp"] <= x2)]
    return len(snps)

In [14]:
def count_clumped_meqtls(chrom, x1, x2):
    common_chr = clumped_meqtls[clumped_meqtls["CHR"] == str(chrom)]
    snps = common_chr[(common_chr["BP"] >= x1) & (common_chr["BP"] <= x2)]
    return len(snps)

### Bin the Genome in 40 Equal Bins, Calculate Normalized Burden of meQTLs in Each Bin

In [33]:
def consecutive_tad_distribution(tads):
    dic = dict()
    active_active_normalized = []
    active_inactive_normalized = []
    inactive_active_normalized = []
    inactive_inactive_normalized = []
    num_bins = 40
    start_boundary = []
    end_boundary = []

    for i in range(num_bins):
        active_active_normalized.append([])
        active_inactive_normalized.append([])
        inactive_active_normalized.append([])
        inactive_inactive_normalized.append([])

    for ind in range(0, len(tads)):
#         print(ind)
#         if ind%105 == 0:
#             print(ind)
        if ind+3>=len(tads):
            break
        if tads.iloc[ind]["chr1"] != tads.iloc[ind+3]["chr1"]:
            continue
        if "Boundary" in tads.iloc[ind]["Label"]:
            continue
        bin_size = (tads.iloc[ind+3]["x2"]-tads.iloc[ind]["x1"])//(num_bins)
        #bin_size = 25000
        temp = tads.iloc[ind]["Label"]
        label1 = tads.iloc[ind]["Label"]
        label2 = tads.iloc[ind+3]["Label"]
        if label1 == "Mixed" or label2 == "Mixed":
            continue
        start = tads.iloc[ind]["x1"]
        inx = 0
        while (start<=tads.iloc[ind+3]["x2"]-bin_size):
            if start>=tads.iloc[ind]["x2"] and start<tads.iloc[ind+2]["x1"]:
                if "Boundary" not in temp:
                    start_boundary.append(inx)
                temp = "Boundary"
            elif start+bin_size<=tads.iloc[ind]["x2"]:
                temp = tads.iloc[ind]["Label"]
            elif start >= tads.iloc[ind+3]["x1"]:
                if temp == "Boundary":
                    end_boundary.append(inx)
                temp = tads.iloc[ind+3]["Label"]
            if label1 == "Active" and label2 == "Active":
                if inx<len(active_active_normalized):
                    active_active_normalized[inx].append(count_clumped_meqtls(tads.iloc[ind]["chr1"], start, start+bin_size)*(1000)/(bin_size))
                else:
                    active_active_normalized.append([])
                    active_active_normalized[inx].append(count_clumped_meqtls(tads.iloc[ind]["chr1"], start, start+bin_size)*(1000)/(bin_size))
            if label1 == "Active" and label2 == "Inactive":
                if inx<len(active_inactive_normalized):
                    active_inactive_normalized[inx].append(count_clumped_meqtls(tads.iloc[ind]["chr1"], start, start+bin_size)*(1000)/(bin_size))
                else:
                    active_inactive_normalized.append([])
                    active_inactive_normalized[inx].append(count_clumped_meqtls(tads.iloc[ind]["chr1"], start, start+bin_size)*(1000)/(bin_size))
            if label1 == "Inactive" and label2 == "Active":
                if inx<len(inactive_active_normalized):
                    inactive_active_normalized[inx].append(count_clumped_meqtls(tads.iloc[ind]["chr1"], start, start+bin_size)*(1000)/(bin_size))
                else:
                    inactive_active_normalized.append([])
                    inactive_active_normalized[inx].append(count_clumped_meqtls(tads.iloc[ind]["chr1"], start, start+bin_size)*(1000)/(bin_size))
            if label1 == "Inactive" and label2 == "Inactive":
                if inx<len(inactive_inactive_normalized):
                    inactive_inactive_normalized[inx].append(count_clumped_meqtls(tads.iloc[ind]["chr1"], start, start+bin_size)*(1000)/(bin_size))
                else:
                    inactive_inactive_normalized.append([])
                    inactive_inactive_normalized[inx].append(count_clumped_meqtls(tads.iloc[ind]["chr1"], start, start+bin_size)*(1000)/(bin_size))
            start = start + bin_size
            inx+=1
    #     break
    return active_active_normalized, active_inactive_normalized, inactive_active_normalized, inactive_inactive_normalized, start_boundary, end_boundary

In [25]:
active_active_normalized_true = []
active_inactive_normalized_true = []
inactive_active_normalized_true = []
inactive_inactive_normalized_true = []
active_active_normalized_true, active_inactive_normalized_true, inactive_active_normalized_true, inactive_inactive_normalized_true, start_boundary, end_boundary = consecutive_tad_distribution(common_TADs_meQTLs)

0
105
210
315
420
525
630
735
840
945
1050
1155
1260
1365
1470
1575
1680
1785
1890
1995
2100
2205
2310
2415
2520
2625
2730
2835
2940
3045
3150


In [26]:
dic_active_active_true = dict()
dic_active_inactive_true = dict()
dic_inactive_active_true = dict()
dic_inactive_inactive_true = dict()
for i in range(len(active_active_normalized_true)):
    dic_active_active_true[i] = np.mean(np.array(active_active_normalized_true[i]))
    dic_active_inactive_true[i] = np.mean(np.array(active_inactive_normalized_true[i]))
    dic_inactive_active_true[i] = np.mean(np.array(inactive_active_normalized_true[i]))
    dic_inactive_inactive_true[i] = np.mean(np.array(inactive_inactive_normalized_true[i]))

### Repeat the Analysis in 100 Randomization Trials

In [ ]:
active_active_normalized_total = []
active_inactive_normalized_total = []
inactive_active_normalized_total = []
inactive_inactive_normalized_total = []
start_boundary = 0
end_boundary = 0
num_trials = 100
for trial in range(num_trials):
    print(trial)
    active_active_normalized = []
    active_inactive_normalized = []
    inactive_active_normalized = []
    inactive_inactive_normalized = []
    random_tads = shuffle_tad_labels(common_TADs_meQTLs)
    active_active_normalized, active_inactive_normalized, inactive_active_normalized, inactive_inactive_normalized, start_boundary, end_boundary = consecutive_tad_distribution(random_tads)
    active_active_normalized_total.append(active_active_normalized)
    active_inactive_normalized_total.append(active_inactive_normalized)
    inactive_active_normalized_total.append(inactive_active_normalized)
    inactive_inactive_normalized_total.append(inactive_inactive_normalized)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44


In [ ]:
dic_active_active_shuffled = dict()
dic_active_inactive_shuffled = dict()
dic_inactive_active_shuffled = dict()
dic_inactive_inactive_shuffled = dict()
for i in range(40):
    temp1 = []
    temp2 = []
    temp3 = []
    temp4 = []
    for j in range(num_trials):
        temp1.append(np.mean(np.array(active_active_normalized_total[j][i])))
        temp2.append(np.mean(np.array(active_inactive_normalized_total[j][i])))
        temp3.append(np.mean(np.array(inactive_active_normalized_total[j][i])))
        temp4.append(np.mean(np.array(inactive_inactive_normalized_total[j][i])))

    dic_active_active_shuffled[i] = np.mean(temp1)
    dic_active_inactive_shuffled[i] = np.mean(temp2)
    dic_inactive_active_shuffled[i] = np.mean(temp3)
    dic_inactive_inactive_shuffled[i] = np.mean(temp4)

### Draw a Line Plot of the Binned Distribution Analysis

In [ ]:
def draw_lineplot(title, data_true, data_shuffled):
    x1 = np.array(list(data_true.keys()))
    y1 = np.array(list(data_true.values()))
    fig1 = plt.figure(figsize=(11, 8))
    ax1 = fig1.add_axes([0.15, 0.15, 0.8, 0.8])
    true_dat = pd.DataFrame()
    true_dat["x1"] = x1
    true_dat["y1"] = y1
    true_dat["avg_y1"] = pd.Series(y1).rolling(4).mean().shift(-2)
    true_dat = true_dat.dropna()
    xnew1 = np.linspace(np.array(true_dat["x1"]).min(), np.array(true_dat["x1"]).max(), 300)
    gfg1 = make_interp_spline(np.array(true_dat["x1"]), np.array(true_dat["avg_y1"]))
    ynew1 = gfg1(xnew1)
    
    x2 = np.array(list(data_shuffled.keys()))
    y2 = np.array(list(data_shuffled.values()))
    shuffled_dat = pd.DataFrame()
    shuffled_dat["x2"] = x2
    shuffled_dat["y2"] = y2
    shuffled_dat["avg_y2"] = pd.Series(y2).rolling(4).mean().shift(-2)
    shuffled_dat = shuffled_dat.dropna()
    xnew2 = np.linspace(np.array(shuffled_dat["x2"]).min(), np.array(shuffled_dat["x2"]).max(), 300)
    gfg2 = make_interp_spline(np.array(shuffled_dat["x2"]), np.array(shuffled_dat["avg_y2"]))
    ynew2 = gfg2(xnew2)
#     yerr1 = []
#     for k in error:
#         yerr1.append(error[k])
#     yerr1 = np.array(yerr1)
#     fig1, ax1 = plt.subplots()
    lst = title.split("-")
    if lst[0] == "Active":
        if lst[0] == lst[2]:
            ax1.axvline(x=np.mean(start_boundary), ymin=0, color='red', linewidth=2, label="Active Boundary")
            ax1.axvline(x=np.mean(end_boundary), ymin=0, color='red', linewidth=2)
        else:
            ax1.axvline(x=np.mean(start_boundary), ymin=0, color='red', linewidth=2, label="Active Boundary")
            ax1.axvline(x=np.mean(end_boundary), ymin=0, color='blue', linewidth=2, label="Inactive Boundary")
    else:
        if lst[0] == lst[2]:
            ax1.axvline(x=np.mean(start_boundary), ymin=0, color='blue', linewidth=2, label="Inactive Boundary")
            ax1.axvline(x=np.mean(end_boundary), ymin=0, color='blue', linewidth=2)
        else:
            ax1.axvline(x=np.mean(start_boundary), ymin=0, color='blue', linewidth=2, label="Inactive Boundary")
            ax1.axvline(x=np.mean(end_boundary), ymin=0, color='red', linewidth=2, label="Active Boundary")
   
    ax1.plot(xnew1, ynew1,linewidth=2, label="True Distribution")
    ax1.plot(xnew2, ynew2,linewidth=2, linestyle='dashed', label="Random Distribution")
    ax1.legend(loc="upper right", fontsize=15)
#     plt.fill_between(xnew1, ynew1-yerr1, ynew1+yerr1)
#     fig = plt.figure(figsize =(15, 7))
    
    ax1.set_xlabel(title, fontsize=20)
    ax1.ticklabel_format(style='plain')
    ax1.set_ylabel("Binned Normalized Burden of meQTLs (Per kb)", fontsize=20)
    ax1.set_title("Burden of Clumped meQTLs across " + title, fontsize=20)
    ax1.tick_params(axis='both', which='major', labelsize=16)
#     fig1.savefig("../plots/"+title+"-clumped-shuffled-Distribution.png")
#     fig1.savefig("../plots/"+title+"-clumped-shuffled-Distribution.pdf")
    plt.show()

In [ ]:
draw_lineplot("Active-Boundary-Active", dic_active_active_true, dic_active_active_shuffled)
draw_lineplot("Active-Boundary-Inactive", dic_active_inactive_true, dic_active_inactive_shuffled)
draw_lineplot("Inactive-Boundary-Active", dic_inactive_active_true, dic_inactive_active_shuffled)
draw_lineplot("Inactive-Boundary-Inactive", dic_inactive_inactive_true, dic_inactive_inactive_shuffled)